In [1]:
import os
HOME = os.getcwd()
try:
    os.mkdir('datasets')
except FileExistsError as file_exist_error:
    print("Cannot create a file when folder 'datasets' already exists")

In [2]:
%cd {HOME}/datasets

/content/datasets


In [8]:
os.environ['HOME'] = "C:\D\Python\portfolio\Projects\pills-detection"

In [9]:
!pip install roboflow

In [10]:
from roboflow import Roboflow
rf = Roboflow(api_key="QvBVrFs8SdlwCX7XOhBR")
project = rf.workspace("seblful").project("pills-detection-s9ywn")
dataset = project.version(19).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [11]:
dataset.location

'/content/datasets/Pills-Detection-19'

Training

In [13]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.4/910.4 kB 12.6 MB/s eta 0:00:00


In [14]:
import ultralytics

ultralytics.checks()

Ultralytics 8.3.64 🚀 Python-3.11.11 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 31.2/107.7 GB disk)


In [15]:
import torch
torch.__version__

'2.5.1+cu121'

In [16]:
# Instantiate device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Current device is {device}")

Current device is cpu


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8s.yaml")  # build a new model from scratch
model = YOLO("yolov8s.pt")  # load a pretrained model (recommended for training)

# Transform model to GPU
# model.to(device)

# Use the model
model.train(data=dataset.location+'\\data.yaml', epochs=100, imgsz=640, batch=8)  # train the model

In [ ]:
metrics = model.val()  # evaluate model performance on the validation set

In [ ]:
results = model("https://upload.wikimedia.org/wikipedia/commons/f/fe/Prozac_pills.jpg")  # predict on an image
success = model.export(format="onnx")  # export the model to ONNX format

In [ ]:
# Detection

%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.4 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[6:16]:
      display(Image(filename=image_path, width=800))
      print("\n")

Deploy model to roboflow

In [ ]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train/")